# 第4章：构建 SegWit 交易

> **参考**: `book/translations/zh-Hans/Chapter 4.md`  
> **代码示例**: `code/chapter04/`  
> **最后更新**: 2025-12-06

---

隔离见证（Segregated Witness，SegWit）通过将签名数据与交易数据分离，从根本上重构了 Bitcoin 交易。本章通过完整的交易实现演示 SegWit 的核心创新，从创建到广播，使用真实测试网数据追踪使 Taproot 成为可能的见证执行模型。


## 4.1 交易可延展性：SegWit 解决的问题

### Legacy 交易结构与 SegWit

传统 Bitcoin 交易将所有数据捆绑在一起进行 TXID 计算，而 SegWit 分离见证数据：

```
Legacy Transaction Structure:
┌─────────────────────────────────────────┐
│ Version │ Inputs │ Outputs │ Locktime   │
│         │ ┌─────┐│         │            │
│         │ │ScSig││         │            │  } All included in TXID
│         │ │     ││         │            │
│         │ └─────┘│         │            │
└─────────────────────────────────────────┘
           ↓
    TXID = SHA256(SHA256(entire_transaction))


SegWit Transaction Structure:
┌─────────────────────────────────────────┐
│ Version │ Inputs │ Outputs │ Locktime   │  } Base Transaction
│         │ ┌─────┐│         │            │
│         │ │Empty││         │            │
│         │ │ScSig││         │            │
│         │ └─────┘│         │            │
└─────────────────────────────────────────┘
                                             } TXID = SHA256(SHA256(base_only))
┌─────────────────────────────────────────┐
│        Witness Data (Separated)         │  } Committed separately
│    ┌─────────────────────────────────┐  │
│    │ Signature │ Public Key         │  │  (For P2WPKH)
│    └─────────────────────────────────┘  │
└─────────────────────────────────────────┘
```

### 可延展性问题演示

在 SegWit 之前，攻击者可以修改签名编码而不影响签名有效性，但会改变 TXID。ECDSA 签名使用的 DER（Distinguished Encoding Rules）格式允许同一签名的多种有效编码。例如：

- **原始签名**：`304402201234567890abcdef...`（71 字节）
- **可延展版本**：`3045022100001234567890abcdef...`（72 字节，带零填充）

两个签名在密码学上相同，都会通过 ECDSA 验证，但它们有不同的字节表示。由于 Legacy Bitcoin 在 TXID 计算中包含整个 scriptSig（包含签名），这些不同的编码为同一经济交易产生不同的交易 ID。

这种可延展性破坏了依赖特定 TXID 的协议，特别是 Lightning Network：

```
Lightning Channel Setup:
Funding TX (TXID_A) → Commitment TX → Timeout TX
                          ↓              ↓
                     References      References
                       TXID_A         TXID_B

If TXID_A changes due to malleability:
→ Commitment TX becomes invalid
→ Timeout TX becomes invalid  
→ Entire channel unusable
```

### Legacy 与 SegWit 代码对比

编程模型的差异突出了架构变化：

**Legacy P2PKH 签名：**


In [1]:
# 示例 1: Legacy vs SegWit 签名对比
# 参考: code/chapter04/01_legacy_vs_segwit_comparison.py

from bitcoinutils.setup import setup
from bitcoinutils.keys import PrivateKey, P2pkhAddress, P2wpkhAddress
from bitcoinutils.transactions import Transaction, TxInput, TxOutput, TxWitnessInput
from bitcoinutils.script import Script
from bitcoinutils.utils import to_satoshis

def legacy_p2pkh_signing():
    """Demonstrates legacy P2PKH transaction signing"""
    print("=" * 60)
    print("LEGACY P2PKH SIGNING")
    print("=" * 60)
    
    setup('testnet')
    
    private_key_wif = 'cPeon9fBsW2BxwJTALj3hGzh9vm8C52Uqsce7MzXGS1iFJkPF4AT'
    sk = PrivateKey(private_key_wif)
    from_addr = P2pkhAddress(sk.get_public_key().get_address().to_string())
    
    print(f"Private Key (WIF): {private_key_wif}")
    print(f"From Address: {from_addr.to_string()}")
    
    previous_locking_script = Script([
        "OP_DUP",
        "OP_HASH160", 
        from_addr.to_hash160(),
        "OP_EQUALVERIFY",
        "OP_CHECKSIG"
    ])
    
    print(f"\nPrevious Locking Script: {previous_locking_script.to_hex()}")
    
    txin = TxInput('5e4a294028ea8cb0e156dac36f4444e2c445c7b393e87301b12818b06cee49e0', 0)
    txout = TxOutput(to_satoshis(0.00000866), P2pkhAddress('myYHJtG3cyoRseuTwvViGHgP2efAvZkYa4').to_script_pub_key())
    tx = Transaction([txin], [txout])
    
    sig = sk.sign_input(tx, 0, previous_locking_script)
    pk = sk.get_public_key().to_hex()
    unlocking_script = Script([sig, pk])
    txin.script_sig = unlocking_script
    
    print(f"\nUnlocking Script (scriptSig): {txin.script_sig.to_hex()}")
    print(f"Signature: {sig[:20]}...{sig[-10:]}")
    print(f"Public Key: {pk}")
    print(f"\n✓ Signature goes in scriptSig (included in TXID)")
    
    return tx


def segwit_p2wpkh_signing():
    """Demonstrates SegWit P2WPKH transaction signing"""
    print("\n" + "=" * 60)
    print("SEGWIT P2WPKH SIGNING")
    print("=" * 60)
    
    setup('testnet')
    
    private_key = PrivateKey('cPeon9fBsW2BxwJTALj3hGzh9vm8C52Uqsce7MzXGS1iFJkPF4AT')
    public_key = private_key.get_public_key()
    from_address = public_key.get_segwit_address()
    
    print(f"Private Key (WIF): {private_key.to_wif()}")
    print(f"From Address: {from_address.to_string()}")
    
    # Real UTXO from testnet (same as in 02_create_segwit_transaction.py)
    utxo_txid = '1454438e6f417d710333fbab118058e2972127bdd790134ab74937fa9dddbc48'
    utxo_vout = 0
    utxo_amount = 1000  # sats
    
    txin = TxInput(utxo_txid, utxo_vout)
    to_address = P2wpkhAddress('tb1qckeg66a6jx3xjw5mrpmte5ujjv3cjrajtvm9r4')
    txout = TxOutput(to_satoshis(0.00000666), to_address.to_script_pub_key())
    
    # CRITICAL: has_segwit=True required for SegWit
    tx = Transaction([txin], [txout], has_segwit=True)
    
    # CRITICAL: Use sign_segwit_input with script_code from public key's address
    script_code = public_key.get_address().to_script_pub_key()
    signature = private_key.sign_segwit_input(
        tx,
        0,
        script_code,
        to_satoshis(utxo_amount / 100000000)
    )
    
    # CRITICAL: ScriptSig must be empty for SegWit
    txin.script_sig = Script([])
    
    # CRITICAL: Use TxWitnessInput wrapper
    tx.witnesses.append(TxWitnessInput([signature, public_key.to_hex()]))
    
    print(f"\nScriptSig: '{txin.script_sig.to_hex() if txin.script_sig else ''}'")
    witness_count = len([signature, public_key.to_hex()]) if tx.witnesses else 0
    print(f"Witness Items: {witness_count}")
    print(f"  [0] Signature: {signature[:20]}...{signature[-10:]}")
    print(f"  [1] Public Key: {public_key.to_hex()}")
    print(f"\n✓ Signature goes in witness (NOT in TXID)")
    print(f"✓ ScriptSig remains empty")
    print(f"✓ Used sign_segwit_input (not sign_input)")
    print(f"✓ Script code from public key's legacy address")
    
    return tx


if __name__ == "__main__":
    legacy_tx = legacy_p2pkh_signing()
    segwit_tx = segwit_p2wpkh_signing()
    
    print("\n" + "=" * 60)
    print("KEY DIFFERENCES")
    print("=" * 60)
    print("Legacy:")
    print("  - Signature in scriptSig")
    print("  - scriptSig included in TXID calculation")
    print("  - Vulnerable to transaction malleability")
    print("\nSegWit:")
    print("  - Signature in witness")
    print("  - scriptSig empty (00)")
    print("  - Witness excluded from TXID calculation")
    print("  - Malleability resistant")


LEGACY P2PKH SIGNING
Private Key (WIF): cPeon9fBsW2BxwJTALj3hGzh9vm8C52Uqsce7MzXGS1iFJkPF4AT
From Address: myYHJtG3cyoRseuTwvViGHgP2efAvZkYa4

Previous Locking Script: 76a914c5b28d6bba91a2693a9b1876bcd3929323890fb288ac

Unlocking Script (scriptSig): 473044022055ffab3f8f53111ce1927d99a507e37808acea32fde0b08bf8ab39b55b8e3dfe0220307637a97d4390488e3d84759f8bd8f84c8733784ef76f8d46db0f860064a574012102898711e6bf63f5cbe1b38c05e89d6c391c59e9f8f695da44bf3d20ca674c8519
Signature: 3044022055ffab3f8f53...0064a57401
Public Key: 02898711e6bf63f5cbe1b38c05e89d6c391c59e9f8f695da44bf3d20ca674c8519

✓ Signature goes in scriptSig (included in TXID)

SEGWIT P2WPKH SIGNING
Private Key (WIF): cPeon9fBsW2BxwJTALj3hGzh9vm8C52Uqsce7MzXGS1iFJkPF4AT
From Address: tb1qckeg66a6jx3xjw5mrpmte5ujjv3cjrajtvm9r4

ScriptSig: ''
Witness Items: 2
  [0] Signature: 3044022015098d26918b...49e33c0301
  [1] Public Key: 02898711e6bf63f5cbe1b38c05e89d6c391c59e9f8f695da44bf3d20ca674c8519

✓ Signature goes in witness (NOT in TXID)


**SegWit P2WPKH 签名：**



## 4.2 创建完整的 SegWit 交易

让我们逐步构建真实的 SegWit 交易，通过实际实现理解 SegWit 如何解决可延展性。

### 交易设置


In [3]:
# 示例 2: 交易设置
# 参考: code/chapter04/02_create_segwit_transaction.py

from bitcoinutils.setup import setup
from bitcoinutils.keys import PrivateKey, P2wpkhAddress
from bitcoinutils.transactions import Transaction, TxInput, TxOutput, TxWitnessInput
from bitcoinutils.utils import to_satoshis
from bitcoinutils.script import Script
def create_segwit_transaction():
    """Creates a complete SegWit transaction step by step"""
    setup('testnet')
    print("=" * 60)
    print("SEGWIT TRANSACTION SETUP")
    print("=" * 60)
    # Private key and public key
    private_key = PrivateKey('cPeon9fBsW2BxwJTALj3hGzh9vm8C52Uqsce7MzXGS1iFJkPF4AT')
    public_key = private_key.get_public_key()
    # CRITICAL: Get script_code from the public key's address
    # This is required for SegWit signing - must derive from the public key
    script_code = public_key.get_address().to_script_pub_key()
    # Addresses
    from_address = P2wpkhAddress('tb1qckeg66a6jx3xjw5mrpmte5ujjv3cjrajtvm9r4')
    to_address = P2wpkhAddress('tb1qckeg66a6jx3xjw5mrpmte5ujjv3cjrajtvm9r4')
    print(f"From: {from_address.to_string()}")
    print(f"To:   {to_address.to_string()}")
    print(f"Script Code (from pubkey): {script_code.to_hex()}")
    # Verify private key matches address
    print(f"\n=== Private Key Verification ===")
    print(f"Private key WIF: {private_key.to_wif()}")
    print(f"Generated address: {public_key.get_segwit_address().to_string()}")
    print(f"Expected address: {from_address.to_string()}")
    print(f"Match: {'✓' if public_key.get_segwit_address().to_string() == from_address.to_string() else '✗'}")
    print("\n" + "=" * 60)
    print("PHASE 1: CREATE UNSIGNED TRANSACTION")
    print("=" * 60)
    # UTXO information
    utxo_txid = '1454438e6f417d710333fbab118058e2972127bdd790134ab74937fa9dddbc48'
    utxo_vout = 0
    utxo_amount = 1000  # sats (from UTXO data)
    # Transaction amounts
    amount_to_send = 666  # sats
    fee = 334  # sats (1000 - 666)
    txin = TxInput(utxo_txid, utxo_vout)


**输出：**
```
From: tb1qckeg66a6jx3xjw5mrpmte5ujjv3cjrajtvm9r4
To:   tb1qckeg66a6jx3xjw5mrpmte5ujjv3cjrajtvm9r4
```

**注意：** 此示例使用了成功广播到测试网的真实交易。交易 TXID 是 `271cf6285479885a5ffa4817412bfcf55e7d2cf43ab1ede06c4332b46084e3e6`，可以在测试网浏览器上查看。


## 4.3 SegWit 交易构建与分析

### 逐步构建交易

让我们逐步构建交易，观察每个阶段的数据结构变化：

### 阶段 1：创建未签名交易


In [4]:
# 示例 3: 创建 SegWit 交易（阶段1和阶段2）
# 参考: code/chapter04/02_create_segwit_transaction.py

from bitcoinutils.setup import setup
from bitcoinutils.keys import PrivateKey, P2wpkhAddress
from bitcoinutils.transactions import Transaction, TxInput, TxOutput, TxWitnessInput
from bitcoinutils.utils import to_satoshis
from bitcoinutils.script import Script

def create_segwit_transaction():
    """Creates a complete SegWit transaction step by step"""
    setup('testnet')
    
    print("=" * 60)
    print("SEGWIT TRANSACTION SETUP")
    print("=" * 60)
    
    # Private key and public key
    private_key = PrivateKey('cPeon9fBsW2BxwJTALj3hGzh9vm8C52Uqsce7MzXGS1iFJkPF4AT')
    public_key = private_key.get_public_key()
    
    # CRITICAL: Get script_code from the public key's address
    # This is required for SegWit signing - must derive from the public key
    script_code = public_key.get_address().to_script_pub_key()
    
    # Addresses
    from_address = P2wpkhAddress('tb1qckeg66a6jx3xjw5mrpmte5ujjv3cjrajtvm9r4')
    to_address = P2wpkhAddress('tb1qckeg66a6jx3xjw5mrpmte5ujjv3cjrajtvm9r4')
    
    print(f"From: {from_address.to_string()}")
    print(f"To:   {to_address.to_string()}")
    print(f"Script Code (from pubkey): {script_code.to_hex()}")
    
    # Verify private key matches address
    print(f"\n=== Private Key Verification ===")
    print(f"Private key WIF: {private_key.to_wif()}")
    print(f"Generated address: {public_key.get_segwit_address().to_string()}")
    print(f"Expected address: {from_address.to_string()}")
    print(f"Match: {'✓' if public_key.get_segwit_address().to_string() == from_address.to_string() else '✗'}")
    
    print("\n" + "=" * 60)
    print("PHASE 1: CREATE UNSIGNED TRANSACTION")
    print("=" * 60)
    
    # UTXO information
    utxo_txid = '1454438e6f417d710333fbab118058e2972127bdd790134ab74937fa9dddbc48'
    utxo_vout = 0
    utxo_amount = 1000  # sats (from UTXO data)
    
    # Transaction amounts
    amount_to_send = 666  # sats
    fee = 334  # sats (1000 - 666)
    
    txin = TxInput(utxo_txid, utxo_vout)
    txout = TxOutput(to_satoshis(amount_to_send / 100000000), to_address.to_script_pub_key())
    
    # CRITICAL: has_segwit=True is required for witness data serialization
    tx = Transaction([txin], [txout], has_segwit=True)
    unsigned_tx = tx.serialize()
    
    print(f"Unsigned TX: {unsigned_tx}")
    print(f"\nTransaction Components:")
    print(f"  Version:      02000000")
    print(f"  Input Count:  01")
    print(f"  TXID:         {utxo_txid}")
    print(f"  VOUT:         {utxo_vout:08x}")
    print(f"  ScriptSig:    00 (empty, 0 bytes)")
    print(f"  Sequence:     fffffffd (RBF enabled)")
    print(f"  Output Count: 01")
    print(f"  Value:        {amount_to_send} sats")
    print(f"  ScriptPubKey: {to_address.to_script_pub_key().to_hex()}")
    print(f"  Locktime:     00000000")
    print(f"\nKey Observations:")
    print(f"  - Standard Bitcoin transaction structure")
    print(f"  - ScriptSig is empty (00) - normal for SegWit")
    print(f"  - No witness data yet")
    
    print("\n" + "=" * 60)
    print("PHASE 2: ADD SEGWIT SIGNATURE")
    print("=" * 60)
    
    # CRITICAL: Use sign_segwit_input (not sign_input)
    # Must provide:
    # 1. script_code: Derived from public key's address (required for SegWit)
    # 2. input_amount: The UTXO amount in satoshis (required for SegWit)
    print(f"Signing with:")
    print(f"  Script Code: {script_code.to_hex()}")
    print(f"  Input Amount: {utxo_amount} sats")
    
    signature = private_key.sign_segwit_input(
        tx,
        0,
        script_code,  # CRITICAL: Must use script_code from public key's address
        to_satoshis(utxo_amount / 100000000)  # CRITICAL: Must provide input amount
    )
    
    # CRITICAL: ScriptSig must be empty for SegWit
    txin.script_sig = Script([])
    
    # CRITICAL: Use TxWitnessInput to wrap witness data
    public_key_hex = public_key.to_hex()
    tx.witnesses.append(TxWitnessInput([signature, public_key_hex]))
    
    print(f"\nScriptSig: '{txin.script_sig.to_hex() if txin.script_sig else ''}' (must be empty)")
    if tx.witnesses:
        # Witness data is stored in the list we passed to TxWitnessInput
        witness_items = [signature, public_key_hex]
        print(f"Witness Items: {len(witness_items)}")
        print(f"  [0] Signature: {signature[:20]}...{signature[-10:]}")
        print(f"  [1] Public Key: {public_key_hex}")
    else:
        print(f"Witness Items: 0")
    
    signed_tx = tx.serialize()
    print(f"\nSigned TX: {signed_tx}")
    
    print(f"\nCritical Changes:")
    print(f"  - ScriptSig remains empty (required for SegWit)")
    print(f"  - Witness data appears (using TxWitnessInput)")
    print(f"  - Transaction becomes longer (added witness section)")
    print(f"  - Used sign_segwit_input (not sign_input)")
    print(f"  - Provided script_code and input_amount (required for SegWit)")
    
    print("\n" + "=" * 60)
    print("TRANSACTION STRUCTURE COMPARISON")
    print("=" * 60)
    print("Before Signing (Phase 1):")
    print("  Standard Bitcoin Transaction Format")
    print("  Total: 84 bytes")
    print("\nAfter Signing (Phase 2):")
    print("  SegWit Transaction Format")
    print("  ├── Version: 02000000")
    print("  ├── Marker: 00 (NEW - SegWit indicator)")
    print("  ├── Flag: 01 (NEW - SegWit version)")
    print("  ├── Input Data (ScriptSig still empty)")
    print("  ├── Output Data")
    print("  ├── Witness Data (NEW - authorization data)")
    print("  └── Locktime: 00000000")
    print("  Total: 191 bytes (added witness section: 82 bytes)")
    print("\nNote: marker/flag (00 01) appear only in serialized form")
    print("      to indicate SegWit and do not participate in txid")
    print("      (they do participate in wtxid)")
    
    return tx, unsigned_tx, signed_tx


if __name__ == "__main__":
    tx, unsigned_tx, signed_tx = create_segwit_transaction()
    
    print("\n" + "=" * 60)
    print("SUMMARY")
    print("=" * 60)
    print("✓ SegWit separates witness data from transaction data")
    print("✓ ScriptSig remains empty for native SegWit")
    print("✓ Witness data excluded from TXID calculation")
    print("✓ Transaction malleability resistance achieved")
    print("\n" + "=" * 60)
    print("VERIFIED ON-CHAIN TRANSACTION")
    print("=" * 60)
    print("✅ This code has been successfully tested and broadcast to testnet")
    print(f"   TXID: 271cf6285479885a5ffa4817412bfcf55e7d2cf43ab1ede06c4332b46084e3e6")
    print(f"   Explorer: https://blockstream.info/testnet/tx/271cf6285479885a5ffa4817412bfcf55e7d2cf43ab1ede06c4332b46084e3e6")
    print(f"   Input: 1,000 sats (V0_P2WPKH)")
    print(f"   Output: 666 sats (V0_P2WPKH)")
    print(f"   Fee: 334 sats")
    print(f"   Status: Confirmed on testnet")


SEGWIT TRANSACTION SETUP
From: tb1qckeg66a6jx3xjw5mrpmte5ujjv3cjrajtvm9r4
To:   tb1qckeg66a6jx3xjw5mrpmte5ujjv3cjrajtvm9r4
Script Code (from pubkey): 76a914c5b28d6bba91a2693a9b1876bcd3929323890fb288ac

=== Private Key Verification ===
Private key WIF: cPeon9fBsW2BxwJTALj3hGzh9vm8C52Uqsce7MzXGS1iFJkPF4AT
Generated address: tb1qckeg66a6jx3xjw5mrpmte5ujjv3cjrajtvm9r4
Expected address: tb1qckeg66a6jx3xjw5mrpmte5ujjv3cjrajtvm9r4
Match: ✓

PHASE 1: CREATE UNSIGNED TRANSACTION
Unsigned TX: 0200000000010148bcdd9dfa3749b74a1390d7bd272197e2588011abfb3303717d416f8e4354140000000000fdffffff019a02000000000000160014c5b28d6bba91a2693a9b1876bcd3929323890fb200000000

Transaction Components:
  Version:      02000000
  Input Count:  01
  TXID:         1454438e6f417d710333fbab118058e2972127bdd790134ab74937fa9dddbc48
  VOUT:         00000000
  ScriptSig:    00 (empty, 0 bytes)
  Sequence:     fffffffd (RBF enabled)
  Output Count: 01
  Value:        666 sats
  ScriptPubKey: 0014c5b28d6bba91a2693a9b1876bcd39

**未签名交易输出：**
`0200000000010148bcdd9dfa3749b74a1390d7bd272197e2588011abfb3303717d416f8e4354140000000000fdffffff019a02000000000000160014c5b28d6bba91a2693a9b1876bcd3929323890fb200000000`

**解析组件：**
```
Version:      02000000
Marker:       00 (SegWit indicator)
Flag:         01 (SegWit version)
Input Count:  01
TXID:         1454438e6f417d710333fbab118058e2972127bdd790134ab74937fa9dddbc48
VOUT:         00000000
ScriptSig:    00 (empty, 0 bytes)
Sequence:     fffffffd (RBF enabled - Replace-By-Fee)
Output Count: 01
Value:        9a02000000000000 (666 sats)
Script Len:   16 (22 bytes)
ScriptPubKey: 0014c5b28d6bba91a2693a9b1876bcd3929323890fb2
Locktime:     00000000
```

**关键观察：**
- 标准 Bitcoin 交易结构
- ScriptSig 为空（`00`）——这对 SegWit 是正常的
- 还没有见证数据

### 阶段 2：添加 SegWit 签名


**阶段 2 输出：**
```
ScriptSig: ''
Witness Items: 2
  [0] Signature: 3044022015098d26918b...49e33c0301
  [1] Public Key: 02898711e6bf63f5cbe1b38c05e89d6c391c59e9f8f695da44bf3d20ca674c8519
Signed TX: 0200000000010148bcdd9dfa3749b74a1390d7bd272197e2588011abfb3303717d416f8e4354140000000000fdffffff019a02000000000000160014c5b28d6bba91a2693a9b1876bcd3929323890fb202473044022015098d26918b46ab36b0d1b50ee502b33d5c5b5257c76bd6d00ccb31452c25ae0220256e82d4df10981f25f91e5273be39fced8fe164434616c94fa48f3549e33c03012102898711e6bf63f5cbe1b38c05e89d6c391c59e9f8f695da44bf3d20ca674c851900000000
```

**已验证的交易：** 此交易已成功广播到测试网。TXID: `271cf6285479885a5ffa4817412bfcf55e7d2cf43ab1ede06c4332b46084e3e6`

**关键变化：**
- ScriptSig 保持为空
- 出现见证数据
- 交易变长（添加了见证部分）

### 交易结构对比

**签名前（阶段 1）：**
```
Standard Bitcoin Transaction Format (with SegWit marker/flag)
├── Version: 02000000
├── Marker: 00 (SegWit indicator)
├── Flag: 01 (SegWit version)
├── Input Count: 01
├── Input Data: 48bcdd9d...00fdffffff (ScriptSig empty)
├── Output Count: 01  
├── Output Data: 9a020000...3890fb2
└── Locktime: 00000000

Total: 84 bytes (base transaction)
```

**签名后（阶段 2）：**
```
SegWit Transaction Format
├── Version: 02000000
├── Marker: 00 (SegWit indicator)
├── Flag: 01 (SegWit version)  
├── Input Count: 01
├── Input Data: 48bcdd9d...00fdffffff (ScriptSig still empty)
├── Output Count: 01
├── Output Data: 9a020000...3890fb2
├── Witness Data: 0247304402...c8519 (NEW - authorization data)
└── Locktime: 00000000

Total: 191 bytes (added witness section: 82 bytes)
```

**注意：** Sequence `0xfffffffd` 表示启用了 RBF（Replace-By-Fee，按费用替换），允许在需要时用更高的费用替换交易。这就是为什么链上浏览器在交易特征中显示 "RBF"。

注意：标记/标志（00 01）仅出现在序列化形式中以指示 SegWit，不参与 txid（它们参与 wtxid）。


### 原始交易解析组件

**带标签组件的完整签名交易：**
```
[VERSION]       02000000
[MARKER]        00      (SegWit indicator)
[FLAG]          01      (SegWit version)
[INPUT_COUNT]   01
[TXID]          1454438e6f417d710333fbab118058e2972127bdd790134ab74937fa9dddbc48
[VOUT]          00000000
[SCRIPTSIG_LEN] 00      (Empty - authorization moved to witness)
[SEQUENCE]      fffffffd
[OUTPUT_COUNT]  01
[VALUE]         9a02000000000000  (666 satoshis)
[SCRIPT_LEN]    16      (22 bytes)
[SCRIPTPUBKEY]  0014c5b28d6bba91a2693a9b1876bcd3929323890fb2
[WITNESS_ITEMS] 02      (2 items: signature + public key)
[SIG_LEN]       47      (71 bytes)
[SIGNATURE]     3044022015098d26918b46ab36b0d1b50ee502b33d5c5b5257c76bd6d00ccb31452c25ae0220256e82d4df10981f25f91e5273be39fced8fe164434616c94fa48f3549e33c0301
[PK_LEN]        21      (33 bytes)
[PUBLIC_KEY]    02898711e6bf63f5cbe1b38c05e89d6c391c59e9f8f695da44bf3d20ca674c8519
[LOCKTIME]      00000000
```


In [5]:
# 示例 4: 解析 SegWit 交易
# 参考: code/chapter04/03_parse_segwit_transaction.py

import struct
from bitcoinutils.setup import setup
from bitcoinutils.keys import PrivateKey, P2wpkhAddress
from bitcoinutils.transactions import Transaction, TxInput, TxOutput, TxWitnessInput
from bitcoinutils.utils import to_satoshis
from bitcoinutils.script import Script


def parse_varint(data, offset):
    """Parse variable-length integer from transaction data"""
    first_byte = data[offset]
    if first_byte < 0xfd:
        return first_byte, offset + 1
    elif first_byte == 0xfd:
        return struct.unpack('<H', data[offset+1:offset+3])[0], offset + 3
    elif first_byte == 0xfe:
        return struct.unpack('<I', data[offset+1:offset+5])[0], offset + 5
    else:  # 0xff
        return struct.unpack('<Q', data[offset+1:offset+9])[0], offset + 9


def parse_segwit_transaction(tx_hex):
    """Parse a SegWit transaction hex string into components"""
    tx_bytes = bytes.fromhex(tx_hex)
    offset = 0
    
    # Version (4 bytes, little-endian)
    version = struct.unpack('<I', tx_bytes[offset:offset+4])[0]
    offset += 4
    
    # Check for SegWit marker and flag
    is_segwit = False
    if offset < len(tx_bytes) and tx_bytes[offset] == 0x00:
        marker = tx_bytes[offset]
        offset += 1
        flag = tx_bytes[offset]
        offset += 1
        is_segwit = True
    else:
        marker = None
        flag = None
    
    # Input count
    input_count, offset = parse_varint(tx_bytes, offset)
    
    inputs = []
    for i in range(input_count):
        # TXID (32 bytes, little-endian, but displayed as big-endian)
        if offset + 32 > len(tx_bytes):
            break
        txid_bytes = tx_bytes[offset:offset+32]
        txid = txid_bytes[::-1].hex()  # Reverse for display (little-endian to big-endian)
        offset += 32
        
        # VOUT (4 bytes, little-endian)
        vout = struct.unpack('<I', tx_bytes[offset:offset+4])[0]
        offset += 4
        
        # ScriptSig length
        script_sig_len, offset = parse_varint(tx_bytes, offset)
        script_sig = tx_bytes[offset:offset+script_sig_len].hex() if script_sig_len > 0 else ''
        offset += script_sig_len
        
        # Sequence (4 bytes, little-endian)
        sequence = struct.unpack('<I', tx_bytes[offset:offset+4])[0]
        offset += 4
        
        inputs.append({
            'txid': txid,
            'vout': vout,
            'script_sig': script_sig,
            'script_sig_len': script_sig_len,
            'sequence': f'{sequence:08x}'
        })
    
    # Output count
    output_count, offset = parse_varint(tx_bytes, offset)
    
    outputs = []
    for i in range(output_count):
        # Check if we have enough bytes
        if offset + 8 > len(tx_bytes):
            break
        # Value (8 bytes, little-endian)
        value = struct.unpack('<Q', tx_bytes[offset:offset+8])[0]
        offset += 8
        
        # Script length
        if offset >= len(tx_bytes):
            break
        script_len, offset = parse_varint(tx_bytes, offset)
        if offset + script_len > len(tx_bytes):
            break
        script_pubkey = tx_bytes[offset:offset+script_len].hex()
        offset += script_len
        
        outputs.append({
            'value': value,
            'value_hex': f'{value:016x}',
            'script_len': script_len,
            'script_pubkey': script_pubkey
        })
    
    # Witness data (if SegWit)
    witnesses = []
    if is_segwit and offset < len(tx_bytes):
        for i in range(input_count):
            if offset >= len(tx_bytes):
                break
            witness_item_count, offset = parse_varint(tx_bytes, offset)
            witness_items = []
            for j in range(witness_item_count):
                if offset >= len(tx_bytes):
                    break
                item_len, offset = parse_varint(tx_bytes, offset)
                if offset + item_len > len(tx_bytes):
                    break
                item_data = tx_bytes[offset:offset+item_len].hex() if item_len > 0 else ''
                offset += item_len
                witness_items.append({
                    'len': item_len,
                    'data': item_data
                })
            witnesses.append(witness_items)
    
    # Locktime (4 bytes, little-endian)
    locktime = 0
    if offset + 4 <= len(tx_bytes):
        locktime = struct.unpack('<I', tx_bytes[offset:offset+4])[0]
        offset += 4
    
    return {
        'version': f'{version:08x}',
        'is_segwit': is_segwit,
        'marker': f'{marker:02x}' if marker is not None else None,
        'flag': f'{flag:02x}' if flag is not None else None,
        'input_count': input_count,
        'inputs': inputs,
        'output_count': output_count,
        'outputs': outputs,
        'witnesses': witnesses,
        'locktime': f'{locktime:08x}',
        'total_size': len(tx_bytes)
    }


def compare_hardcoded_vs_actual():
    """Compare hardcoded transaction structure with actual parsed transaction"""
    setup('testnet')
    
    print("=" * 70)
    print("REAL SEGWIT TRANSACTION PARSING")
    print("=" * 70)
    
    # Use the REAL transaction from 02_create_segwit_transaction.py
    # This is the actual transaction that was broadcast to testnet
    private_key = PrivateKey('cPeon9fBsW2BxwJTALj3hGzh9vm8C52Uqsce7MzXGS1iFJkPF4AT')
    public_key = private_key.get_public_key()
    from_address = public_key.get_segwit_address()
    to_address = P2wpkhAddress('tb1qckeg66a6jx3xjw5mrpmte5ujjv3cjrajtvm9r4')
    
    # Real UTXO from testnet
    utxo_txid = '1454438e6f417d710333fbab118058e2972127bdd790134ab74937fa9dddbc48'
    utxo_vout = 0
    utxo_amount = 1000  # sats
    
    txin = TxInput(utxo_txid, utxo_vout)
    txout = TxOutput(to_satoshis(0.00000666), to_address.to_script_pub_key())
    
    # CRITICAL: has_segwit=True required
    tx = Transaction([txin], [txout], has_segwit=True)
    
    print("\n" + "=" * 70)
    print("PHASE 1: UNSIGNED TRANSACTION")
    print("=" * 70)
    
    unsigned_tx = tx.serialize()
    parsed_unsigned = parse_segwit_transaction(unsigned_tx)
    
    print(f"\nGenerated Transaction Hex:")
    print(f"  {unsigned_tx}")
    print(f"\nActual Parsed Components:")
    print(f"  Version:      {parsed_unsigned['version']}")
    print(f"  Is SegWit:    {parsed_unsigned['is_segwit']}")
    if parsed_unsigned['is_segwit']:
        print(f"  Marker:       {parsed_unsigned['marker']}")
        print(f"  Flag:         {parsed_unsigned['flag']}")
    print(f"  Input Count:  {parsed_unsigned['input_count']:02x}")
    
    if parsed_unsigned['inputs']:
        inp = parsed_unsigned['inputs'][0]
        print(f"  TXID:         {inp['txid']}")
        print(f"  VOUT:         {inp['vout']:08x} ({inp['vout']})")
        print(f"  ScriptSig:    {inp['script_sig'] if inp['script_sig'] else '(empty)'} (len: {inp['script_sig_len']})")
        print(f"  Sequence:     {inp['sequence']}")
    
    print(f"  Output Count: {parsed_unsigned['output_count']:02x}")
    
    if parsed_unsigned['outputs']:
        out = parsed_unsigned['outputs'][0]
        print(f"  Value:        {out['value_hex']} ({out['value']} satoshis)")
        print(f"  Script Len:   {out['script_len']:02x} ({out['script_len']} bytes)")
        print(f"  ScriptPubKey: {out['script_pubkey']}")
    
    print(f"  Locktime:     {parsed_unsigned['locktime']}")
    print(f"  Total Size:   {parsed_unsigned['total_size']} bytes")
    
    print("\n" + "=" * 70)
    print("REAL TRANSACTION DATA")
    print("=" * 70)
    print("  ✅ This is the actual transaction structure from testnet")
    print(f"  Input UTXO:   {utxo_txid}:{utxo_vout}")
    print(f"  Input Amount: {utxo_amount} sats")
    print(f"  Output Amount: 666 sats")
    print(f"  Fee:          334 sats")
    
    print("\n" + "=" * 70)
    print("PHASE 2: SIGNED TRANSACTION WITH WITNESS")
    print("=" * 70)
    
    # CRITICAL: Use sign_segwit_input with script_code and input amount
    script_code = public_key.get_address().to_script_pub_key()
    signature = private_key.sign_segwit_input(
        tx,
        0,
        script_code,
        to_satoshis(utxo_amount / 100000000)
    )
    
    # Set empty scriptSig and add witness
    txin.script_sig = Script([])
    tx.witnesses.append(TxWitnessInput([signature, public_key.to_hex()]))
    
    signed_tx = tx.serialize()
    parsed_signed = parse_segwit_transaction(signed_tx)
    
    print(f"\nGenerated Transaction Hex:")
    print(f"  {signed_tx[:100]}...")
    print(f"\nActual Parsed Components:")
    print(f"  Version:      {parsed_signed['version']}")
    print(f"  Is SegWit:    {parsed_signed['is_segwit']}")
    if parsed_signed['is_segwit']:
        print(f"  Marker:       {parsed_signed['marker']}")
        print(f"  Flag:         {parsed_signed['flag']}")
    
    if parsed_signed['inputs']:
        inp = parsed_signed['inputs'][0]
        print(f"  TXID:         {inp['txid']}")
        print(f"  VOUT:         {inp['vout']:08x}")
        print(f"  ScriptSig:    {inp['script_sig'] if inp['script_sig'] else '(empty)'} (len: {inp['script_sig_len']})")
        print(f"  Sequence:     {inp['sequence']}")
    
    if parsed_signed['outputs']:
        out = parsed_signed['outputs'][0]
        print(f"  Value:        {out['value_hex']} ({out['value']} satoshis)")
        print(f"  Script Len:   {out['script_len']:02x} ({out['script_len']} bytes)")
        print(f"  ScriptPubKey: {out['script_pubkey']}")
    
    if parsed_signed['witnesses']:
        witness = parsed_signed['witnesses'][0]
        print(f"  Witness Items: {len(witness)}")
        for i, item in enumerate(witness):
            print(f"    [{i}] Length: {item['len']} bytes")
            print(f"    [{i}] Data:    {item['data'][:40]}...{item['data'][-20:] if len(item['data']) > 60 else item['data']}")
    
    print(f"  Locktime:     {parsed_signed['locktime']}")
    print(f"  Total Size:   {parsed_signed['total_size']} bytes")
    
    print("\n" + "=" * 70)
    print("REAL ON-CHAIN TRANSACTION")
    print("=" * 70)
    print("✅ This transaction was successfully broadcast to testnet")
    print(f"   TXID: 271cf6285479885a5ffa4817412bfcf55e7d2cf43ab1ede06c4332b46084e3e6")
    print(f"   Explorer: https://blockstream.info/testnet/tx/271cf6285479885a5ffa4817412bfcf55e7d2cf43ab1ede06c4332b46084e3e6")
    print(f"   Input: 1,000 sats (V0_P2WPKH)")
    print(f"   Output: 666 sats (V0_P2WPKH)")
    print(f"   Fee: 334 sats")
    print(f"   Status: Confirmed on testnet")
    print("\nThis parsing demonstrates:")
    print("  - How to parse SegWit transaction hex strings manually")
    print("  - Real transaction structure analysis")
    print("  - Witness data extraction from actual on-chain data")
    print("  - Byte-level encoding of SegWit transactions")
    
    # Also parse the actual on-chain transaction hex
    print("\n" + "=" * 70)
    print("PARSING ACTUAL ON-CHAIN TRANSACTION HEX")
    print("=" * 70)
    
    # Real signed transaction hex from blockchain
    real_signed_tx_hex = "0200000000010148bcdd9dfa3749b74a1390d7bd272197e2588011abfb3303717d416f8e4354140000000000fdffffff019a02000000000000160014c5b28d6bba91a2693a9b1876bcd3929323890fb202473044022015098d26918b46ab36b0d1b50ee502b33d5c5b5257c76bd6d00ccb31452c25ae0220256e82d4df10981f25f91e5273be39fced8fe164434616c94fa48f3549e33c03012102898711e6bf63f5cbe1b38c05e89d6c391c59e9f8f695da44bf3d20ca674c851900000000"
    parsed_real = parse_segwit_transaction(real_signed_tx_hex)
    
    print(f"\nOn-Chain Transaction Hex (first 100 chars):")
    print(f"  {real_signed_tx_hex[:100]}...")
    print(f"\nParsed Components from On-Chain Data:")
    print(f"  Version:      {parsed_real['version']}")
    print(f"  Is SegWit:    {parsed_real['is_segwit']}")
    if parsed_real['is_segwit']:
        print(f"  Marker:       {parsed_real['marker']}")
        print(f"  Flag:         {parsed_real['flag']}")
    print(f"  Input Count:  {parsed_real['input_count']}")
    if parsed_real['inputs']:
        inp = parsed_real['inputs'][0]
        print(f"  TXID:         {inp['txid']}")
        print(f"  VOUT:         {inp['vout']}")
        print(f"  ScriptSig:    {inp['script_sig'] if inp['script_sig'] else '(empty)'}")
    if parsed_real['outputs']:
        out = parsed_real['outputs'][0]
        print(f"  Value:        {out['value']} satoshis")
        print(f"  ScriptPubKey: {out['script_pubkey']}")
    if parsed_real['witnesses']:
        witness = parsed_real['witnesses'][0]
        print(f"  Witness Items: {len(witness)}")
        for i, item in enumerate(witness):
            print(f"    [{i}] Length: {item['len']} bytes")
            if item['len'] > 0:
                print(f"    [{i}] Data:    {item['data'][:40]}...{item['data'][-20:] if len(item['data']) > 60 else item['data']}")
    print(f"  Total Size:   {parsed_real['total_size']} bytes")


if __name__ == "__main__":
    compare_hardcoded_vs_actual()


REAL SEGWIT TRANSACTION PARSING

PHASE 1: UNSIGNED TRANSACTION

Generated Transaction Hex:
  0200000000010148bcdd9dfa3749b74a1390d7bd272197e2588011abfb3303717d416f8e4354140000000000fdffffff019a02000000000000160014c5b28d6bba91a2693a9b1876bcd3929323890fb200000000

Actual Parsed Components:
  Version:      00000002
  Is SegWit:    True
  Marker:       00
  Flag:         01
  Input Count:  01
  TXID:         1454438e6f417d710333fbab118058e2972127bdd790134ab74937fa9dddbc48
  VOUT:         00000000 (0)
  ScriptSig:    (empty) (len: 0)
  Sequence:     fffffffd
  Output Count: 01
  Value:        000000000000029a (666 satoshis)
  Script Len:   16 (22 bytes)
  ScriptPubKey: 0014c5b28d6bba91a2693a9b1876bcd3929323890fb2
  Locktime:     00000000
  Total Size:   84 bytes

REAL TRANSACTION DATA
  ✅ This is the actual transaction structure from testnet
  Input UTXO:   1454438e6f417d710333fbab118058e2972127bdd790134ab74937fa9dddbc48:0
  Input Amount: 1000 sats
  Output Amount: 666 sats
  Fee:          

## 4.4 P2WPKH 栈执行分析

现在让我们使用真实交易数据追踪完整的脚本执行。

### 交易组件

**锁定脚本（ScriptPubKey）：**
```
0014c5b28d6bba91a2693a9b1876bcd3929323890fb2
```

**解析：**
- `00`：OP_0（见证版本 0）
- `14`：推送 20 字节
- `c5b28d6bba91a2693a9b1876bcd3929323890fb2`：公钥哈希（20 字节）

**见证栈（来自真实交易）：**
- 项 0：`3044022015098d26918b46ab36b0d1b50ee502b33d5c5b5257c76bd6d00ccb31452c25ae0220256e82d4df10981f25f91e5273be39fced8fe164434616c94fa48f3549e33c0301`（签名，71 字节）
- 项 1：`02898711e6bf63f5cbe1b38c05e89d6c391c59e9f8f695da44bf3d20ca674c8519`（公钥，33 字节）

### SegWit 执行模型

Bitcoin Core 识别 OP_0 <20-bytes> 模式，将 P2WPKH 作为 P2PKH 的等价物执行：

**有效脚本**：`OP_DUP OP_HASH160 <pubkey_hash> OP_EQUALVERIFY OP_CHECKSIG`

### 栈执行追踪

**初始状态：**
```
│ (empty)                                  │
└─────────────────────────────────────────┘
```

### 1. 将见证项加载到栈

```
│ 02898711e6bf...c8519 (public_key)       │
│ 304402201509...33c0301 (signature)      │
└─────────────────────────────────────────┘
```

### 2. 执行 ScriptPubKey：OP_0 <pubkey_hash>

#### 2a. OP_0：推送见证版本
```
│ 00 (witness_version)                     │
│ 02898711e6bf...c8519 (public_key)       │
│ 304402201509...33c0301 (signature)      │
└─────────────────────────────────────────┘
```

#### 2b. PUSH 公钥哈希：来自脚本的预期哈希
```
│ c5b28d6bba91...890fb2 (expected_hash)  │
│ 00 (witness_version)                     │
│ 02898711e6bf...c8519 (public_key)       │
│ 304402201509...33c0301 (signature)      │
└─────────────────────────────────────────┘
```

### 3. SegWit 解释器：P2WPKH 模式识别

**Bitcoin Core 检测 OP_0 <20-bytes> 为 P2WPKH 并执行 P2PKH 风格的验证：**

这是 SegWit 优雅设计的体现。当 Bitcoin Core 遇到匹配模式 `OP_0 <20-byte-data>` 的 scriptPubKey 时，它识别这是具有 20 字节见证程序的版本 0 见证程序（P2WPKH）。

**模式识别过程：**
1. **版本检测**：`OP_0` 表示见证版本 0
2. **程序长度检查**：20 字节表示 P2WPKH 格式
3. **执行模式切换**：Bitcoin Core 从正常脚本执行切换到见证程序执行
4. **等价脚本构建**：解释器将见证程序视为执行：`OP_DUP OP_HASH160 <20-byte-pubkey-hash> OP_EQUALVERIFY OP_CHECKSIG`

**为什么这很重要：**
这种模式识别至关重要，因为它允许 SegWit 在引入新脚本语义的同时保持完全向后兼容。Legacy 节点看到 `OP_0 <20-bytes>` 并认为它是任何人都可以花费（因为 OP_0 推送空值，使脚本评估为 TRUE）。SegWit 节点识别该模式并应用见证验证规则。

**执行上下文切换：**
一旦识别模式，Bitcoin Core：
- 加载见证栈项（签名和公钥）
- 构建等价的 P2PKH 脚本执行环境
- 验证公钥哈希到 20 字节见证程序
- 针对交易和公钥验证签名

这种模式识别框架使 Taproot 的 OP_1 程序成为可能——相同的架构原则扩展到具有 32 字节数据的版本 1 见证程序。

#### 3a. OP_DUP：复制公钥
```
│ 02898711e6bf...c8519 (public_key)       │
│ 02898711e6bf...c8519 (public_key)       │
│ 304402201509...33c0301 (signature)      │
└─────────────────────────────────────────┘
```

#### 3b. OP_HASH160：哈希公钥
```
│ c5b28d6bba91...890fb2 (computed_hash)  │
│ 02898711e6bf...c8519 (public_key)       │
│ 304402201509...33c0301 (signature)      │
└─────────────────────────────────────────┘
```
**（Hash160 = RIPEMD160(SHA256(public_key))）**
在 BIP143 中，签名消息中使用的 P2WPKH scriptCode 正是 P2PKH 模板：`OP_DUP OP_HASH160 <20-byte-hash> OP_EQUALVERIFY OP_CHECKSIG`。这就是为什么我们从 `public_key.get_address().to_script_pub_key()`（legacy 格式：`76a914c5b28d6bba91a2693a9b1876bcd3929323890fb288ac`）派生 `script_code`，而不是从 SegWit 地址派生。

#### 3c. PUSH 预期哈希：来自见证程序
```
│ c5b28d6bba91...890fb2 (expected_hash)  │
│ c5b28d6bba91...890fb2 (computed_hash)  │
│ 02898711e6bf...c8519 (public_key)       │
│ 304402201509...33c0301 (signature)      │
└─────────────────────────────────────────┘
```

#### 3d. OP_EQUALVERIFY：验证哈希匹配
```
│ 02898711e6bf...c8519 (public_key)       │
│ 304402201509...33c0301 (signature)      │
└─────────────────────────────────────────┘
```
**（哈希验证：computed_hash == expected_hash ✓）**

#### 3e. OP_CHECKSIG：最终签名验证
```
│ 1 (TRUE)                                 │
└─────────────────────────────────────────┘
```
**（针对交易数据的 ECDSA 签名验证 ✓）**

### 执行结果：成功

P2WPKH 花费已授权：
- ✅ 见证版本有效（0）
- ✅ 公钥哈希匹配见证程序
- ✅ 签名验证通过
- ✅ 交易可延展性抗性（TXID 排除见证）

SegWit 引入两个标识符：txid（基础交易的哈希，排除见证）和 wtxid（包含见证）。矿工通过 coinbase 中的见证承诺（wtxids 的 Merkle 根）提交区块的见证数据。


## 4.5 SegWit 到 Taproot 的演化

SegWit 通过三个关键创新建立了使 Taproot 成为可能的架构基础：

### 见证版本框架
```
Version 0: P2WPKH (OP_0 <20-bytes>) and P2WSH (OP_0 <32-bytes>)
Version 1: P2TR (OP_1 <32-bytes>) - Taproot
```

### 可延展性抗性
稳定的交易 ID 支持：
- Lightning Network 通道
- 复杂的预签名交易链
- 可靠的 Layer 2 协议

### 经济激励

SegWit 引入基于权重的费用计算：

```
Transaction Weight = (Base Size × 4) + Witness Size
Virtual Size = Weight ÷ 4
```
直觉：见证字节按 1 权重单位/字节收费，而基础字节为 4 wu/字节。节省取决于有多少授权数据移动到见证（取决于结构，不是固定的 25%/75%）。

**通过分离实现空间效率：**

在 Legacy 交易中，2-of-3 多重签名需要大的 scriptSig：
```
scriptSig: <empty> <sig1> <sig2> <redeemScript>
Total: ~300 bytes in scriptSig (counted at full weight)
```

在 SegWit P2WSH 中，相同的授权移动到见证：
```
scriptSig: <empty> (0 bytes)
witness: <empty> <sig1> <sig2> <witnessScript>
Total: ~300 bytes in witness (75% discount)
```

这种架构变化意味着复杂脚本在经济上变得可行。SegWit 多重签名交易支付的费用比其 Legacy 等价物大约少 25%，而简单的单签名交易节省较少。

**Taproot 放大：**
SegWit 的经济框架为 Taproot 更大的效率奠定了基础。Taproot 通过密钥聚合的单签名链上可以使复杂的多方安排与单签名交易一样便宜，充分利用 SegWit 建立的见证折扣结构。


## 章节总结

本章通过完整的交易实现演示了 SegWit 的核心创新：

**见证结构**：将签名数据与交易逻辑分离，为 Taproot 的脚本树和通过密钥聚合的单签名链上创建基础。

**可延展性抗性**：稳定的交易 ID 支持 Taproot 通过更高效的授权方案优化的 Layer 2 生态系统。

**栈执行模型**：SegWit 解释器对见证程序的模式识别为 Taproot 的 OP_1 执行模型提供模板。

**经济框架**：权重单位折扣为高级脚本设计创造激励，Taproot 通过签名聚合和脚本树效率最大化这些设计。

理解 SegWit 的见证架构和执行模型对于掌握 Taproot 至关重要，因为 P2TR 直接建立在这些基础之上，同时添加 Schnorr 签名、密钥聚合和 Merkle 脚本树。

在下一章中，我们将探索 Schnorr 签名如何提供使 Taproot 的密钥聚合和签名效率成为可能的数学原语，在 SegWit 的见证架构基础上创建 Bitcoin 最先进的授权方案。
